## 05 Merge Interim Data of all Players (FBREF and Transfermarkt)

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import sys
sys.path.append("..")
from src.preprocessing.merge_player_data import run_merge_pipeline

merge_result: tuple = run_merge_pipeline("Valencia CF", ["2223", "2324", "2425"])
df_all_players = merge_result[0] if isinstance(merge_result, tuple) else merge_result

print(f"Merge completed successfully!")
print(f"Total merged rows: {df_all_players.shape[0]}")
print(f"Total columns: {df_all_players.shape[1]}")

Attempting fuzzy matching for 51 unmatched players...
Fuzzy match: 'nicolás gonzález' -> 'nico gonzález' (score: 92)
Fuzzy match: 'javier guerra' -> 'javi guerra' (score: 92)
Fuzzy match: 'jesus vazquez' -> 'jesús vázquez' (score: 92)
Fuzzy match: 'emilio bernard' -> 'emilio bernad' (score: 96)
Fuzzy match: 'charlie perez' -> 'charlie pérez' (score: 96)
Fuzzy match: 'javier guerra' -> 'javi guerra' (score: 92)
Fuzzy match: 'jesus vazquez' -> 'jesús vázquez' (score: 92)
Fuzzy match: 'pablo gozalbez' -> 'pablo gozálbez' (score: 96)
Fuzzy match: 'marco camús' -> 'marco camus' (score: 95)
Fuzzy match: 'javier guerra' -> 'javi guerra' (score: 92)
Fuzzy match: 'jesus vazquez' -> 'jesús vázquez' (score: 92)
Dropped Nation_* columns: ['Nation_player_shooting', 'Nation_player_passing', 'Nation_player_passing_types', 'Nation_player_gca', 'Nation_player_defense', 'Nation_player_possession']
Dropped Age columns: ['Age_player_shooting', 'Age_player_passing', 'Age_player_passing_types', 'Age_player_

In [3]:
total_with_market_value = df_all_players['MarketValueEuro'].notna().sum()
print(f"Rows with market value data: {total_with_market_value}/{len(df_all_players)} ({total_with_market_value/len(df_all_players)*100:.1f}%)")

Rows with market value data: 80/114 (70.2%)


In [4]:
print("\nSample merged data:")
sample_cols = ['Player', 'Season', 'MarketValueEuro', 'Age_x', 'Position']
available_cols = [col for col in sample_cols if col in df_all_players.columns]
display(df_all_players[available_cols].head(20))


Sample merged data:


,Player,Season,MarketValueEuro,Position
21,Alberto Marí,2223,800000.0,Centre-Forward
43,Alberto Marí,2324,1500000.0,Centre-Forward
109,Alberto Marí,2425,NaN,NaN
18,Carlos Soler,2223,25000000.0,Central Midfield
12,Cenk Özkacar,2223,5000000.0,Centre-Back
35,Cenk Özkacar,2324,4000000.0,Centre-Back
117,Cenk Özkacar,2425,NaN,NaN
78,Charlie Perez,2223,25000.0,Goalkeeper
19,Cristhian Mosquera,2223,1000000.0,Centre-Back
31,Cristhian Mosquera,2324,30000000.0,Centre-Back


In [5]:
# Group by player and sort by season to see all seasons for each player with missing market values
players_with_missing_market_values = df_all_players[df_all_players['MarketValueEuro'].isna()].sort_values(['Player', 'Season'])
display(players_with_missing_market_values[['Player', 'Season', 'Age', 'Position']].head(40))

,Player,Season,Age,Position
109,Alberto Marí,2425,23,NaN
117,Cenk Özkacar,2425,23,NaN
76,Cristian,2223,24,NaN
90,Cristian,2324,25,NaN
101,Dani Gómez,2425,26,NaN
108,David Otorbi,2425,16,NaN
70,Domingos André Ribeiro Almeida,2223,22,NaN
86,Domingos André Ribeiro Almeida,2324,23,NaN
97,Domingos André Ribeiro Almeida,2425,24,NaN
74,Francisco Perez,2223,19,NaN


In [6]:
# Remove system/total rows
df_clean = df_all_players[~df_all_players['Player'].str.contains('Total|Squad|Opponent', case=False)]

# Only players with market values
df_with_market_values = df_all_players[df_all_players['MarketValueEuro'].notna()]

In [7]:
df_with_market_values

,Player,Season,Nation,Nationality,Pos,Position,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,xA,A-xAG,KP,1/3,PPA,CrsPA,Live,Dead,TB,Sw,Crs,TI,CK,In,Out,Str,Off,Blocks,SCA,SCA90,PassLive,PassDead,TO,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,TO.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def_3rd,Mid_3rd,Att_3rd,Tkl.1,Tkl%,Lost,Pass,Int,Tkl+Int,Clr,Err,Touches,Def_Pen,Att_Pen,Succ,Succ%,Tkld,Tkld%,Carries,CPA,Mis,Dis,Rec,MarketValueEuro,Current club
21,Alberto Marí,2223,es ESP,Spain,"FW,MF",Centre-Forward,21,5,0,86.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.2,0.2,0.0,0.2,1.0,1.0,6.0,1.05,0.00,1.05,1.05,1.05,0.25,0.00,0.25,0.25,0.25,2.0,1.0,50.0,2.09,1.05,0.50,1.00,10.7,0.0,0.12,0.8,0.8,8.0,20.0,40.0,102.0,7.0,7.0,12.0,58.3,1.0,4.0,25.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,30.0,0.0,7.0,1.0,25.0,3.0,75.0,21.0,1.0,2.0,1.0,26.0,800000.0,Real Zaragoza
43,Alberto Marí,2324,es ESP,Spain,FW,Centre-Forward,22,16,2,276.0,3.1,1.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,1.8,1.2,0.9,2.0,2.0,6.0,13.0,0.33,0.33,0.65,0.00,0.33,0.60,0.28,0.88,0.39,0.67,6.0,2.0,33.3,1.96,0.65,0.00,0.00,11.4,0.0,0.20,-0.8,-1.2,38.0,61.0,62.3,498.0,112.0,22.0,30.0,73.3,11.0,16.0,68.8,2.0,2.0,100.0,0.9,0.1,6.0,3.0,1.0,0.0,56.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,8.0,2.62,6.0,0.0,0.0,1.0,0.0,3.0,0.98,1.0,0.0,0.0,1.0,1.0,0.0,4.0,2.0,0.0,1.0,3.0,1.0,100.0,0.0,3.0,0.0,4.0,5.0,0.0,102.0,6.0,15.0,3.0,60.0,2.0,40.0,44.0,2.0,11.0,3.0,62.0,1500000.0,Real Zaragoza
18,Carlos Soler,2223,es ESP,Spain,MF,Central Midfield,25,3,3,242.0,2.7,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.9,0.2,0.2,0.4,3.0,13.0,7.0,0.37,0.00,0.37,0.00,0.00,0.35,0.08,0.43,0.06,0.14,4.0,1.0,25.0,1.49,0.37,0.00,0.00,30.0,3.0,0.04,0.1,-0.2,92.0,123.0,74.8,1388.0,378.0,52.0,59.0,88.1,23.0,28.0,82.1,9.0,24.0,37.5,0.2,-0.2,4.0,5.0,4.0,0.0,102.0,21.0,0.0,1.0,16.0,0.0,11.0,5.0,5.0,0.0,0.0,3.0,6.0,2.23,5.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,2.0,66.7,1.0,1.0,0.0,2.0,2.0,0.0,143.0,2.0,4.0,1.0,50.0,0.0,0.0,83.0,0.0,2.0,1.0,98.0,25000000.0,West Ham United
12,Cenk Özkacar,2223,tr TUR,Türkiye,DF,Centre-Back,21,17,14,1365.0,15.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.8,0.8,0.2,1.0,23.0,61.0,2.0,0.00,0.00,0.00,0.00,0.00,0.05,0.02,0.07,0.05,0.07,11.0,2.0,18.2,0.73,0.13,0.00,0.00,12.8,0.0,0.07,-0.8,-0.8,646.0,769.0,84.0,12014.0,4500.0,274.0,315.0,87.0,308.0,341.0,90.3,60.0,91.0,65.9,0.3,-0.2,4.0,54.0,3.0,1.0,744.0,23.0,0.0,16.0,3.0,2.0,0.0,0.0,0.0,0.0,2.0,8.0,18.0,1.19,15.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,27.0,20.0,17.0,10.0,0.0,14.0,73.7,5.0,12.0,12.0,39.0,47.0,4.0,908.0,85.0,16.0,10.0,100.0,0.0,0.0,522.0,0.0,8.0,4.0,549.0,5000000.0,Real Valladolid CF
35,Cenk Özkacar,2324,tr TUR,Türkiye,DF,Centre-Back,22,23,18,1598.0,17.8,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.3,0.3,0.1,0.3,17.0,56.0,5.0,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.02,0.01,0.02,6.0,3.0,50.0,0.34,0.17,0.00,0.00,14.3,0.0,0.04,-0.3,-0.3,698.0,860.0,81.2,13108.0,4662.0,265.0,295.0,89.8,364.0,398.0,91.5,58.0,135.0,43.0,0.3,-0.1,1.0,48.0,3.0,0.0,810.0,48.0,1.0,9.0,6.0,21.0,0.0,0.0,0.0,0.0,2.0,15.0,13.0,0.73,12.0,1.0,0.0,0.0,0.0,2.0,0.11,2.0,0.0,0.0,0.0,0.0,0.0,26.0,15.0,22.0,4.0,0.0,15.0,65.2,8.0,11.0,18.0,44.0,84.0,1.0,1035.0,109.0,12.0,3.0,37.5,4.0,50.0,532.0,0.0,4.0,3.0,617.0,4000000.0,Real Valladolid CF
78,Charlie Perez,2223,es ESP,Spain,GK,Goalkeeper,20,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

NOTE: Current issues: 
- Nation and Nationality contain different values -----> Let's use 'Position' it's cleaner
- Pos and Position are not the same and we have in Pos sometimes multiple values -----> By keeping only last 3 characters in 'Pos' the data is cleaner and compact
- We have new players with only one season or missing values -----> Let's skip them then, it's best to look at players with 2-3+ seasons

In [8]:
#### Let's use this column instead of 'Pos' since it's cleaner
df_with_market_values['Position'].value_counts()

Position
Centre-Back           15
Centre-Forward        13
Goalkeeper            10
Central Midfield       9
Right-Back             7
Right Winger           6
Defensive Midfield     5
Attacking Midfield     5
Left Winger            5
Left-Back              4
Second Striker         1
Name: count, dtype: int64

In [9]:
df_with_market_values['Nation'].value_counts().head(3)

Nation
es ESP    47
fr FRA     4
br BRA     4
Name: count, dtype: int64

In [10]:
df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]

/var/folders/1h/14p077t95lbcy6x5tc6bqx3h0000gn/T/ipykernel_53065/3401751002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]


In [11]:
print(df_with_market_values.shape)
df_with_market_values.head(10)

(80, 122)


,Player,Season,Nation,Nationality,Pos,Position,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,xA,A-xAG,KP,1/3,PPA,CrsPA,Live,Dead,TB,Sw,Crs,TI,CK,In,Out,Str,Off,Blocks,SCA,SCA90,PassLive,PassDead,TO,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,TO.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def_3rd,Mid_3rd,Att_3rd,Tkl.1,Tkl%,Lost,Pass,Int,Tkl+Int,Clr,Err,Touches,Def_Pen,Att_Pen,Succ,Succ%,Tkld,Tkld%,Carries,CPA,Mis,Dis,Rec,MarketValueEuro,Current club
21,Alberto Marí,2223,ESP,Spain,"FW,MF",Centre-Forward,21,5,0,86.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.2,0.2,0.0,0.2,1.0,1.0,6.0,1.05,0.00,1.05,1.05,1.05,0.25,0.00,0.25,0.25,0.25,2.0,1.0,50.0,2.09,1.05,0.5,1.0,10.7,0.0,0.12,0.8,0.8,8.0,20.0,40.0,102.0,7.0,7.0,12.0,58.3,1.0,4.0,25.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,30.0,0.0,7.0,1.0,25.0,3.0,75.0,21.0,1.0,2.0,1.0,26.0,800000.0,Real Zaragoza
43,Alberto Marí,2324,ESP,Spain,FW,Centre-Forward,22,16,2,276.0,3.1,1.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,1.8,1.2,0.9,2.0,2.0,6.0,13.0,0.33,0.33,0.65,0.00,0.33,0.60,0.28,0.88,0.39,0.67,6.0,2.0,33.3,1.96,0.65,0.0,0.0,11.4,0.0,0.20,-0.8,-1.2,38.0,61.0,62.3,498.0,112.0,22.0,30.0,73.3,11.0,16.0,68.8,2.0,2.0,100.0,0.9,0.1,6.0,3.0,1.0,0.0,56.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,8.0,2.62,6.0,0.0,0.0,1.0,0.0,3.0,0.98,1.0,0.0,0.0,1.0,1.0,0.0,4.0,2.0,0.0,1.0,3.0,1.0,100.0,0.0,3.0,0.0,4.0,5.0,0.0,102.0,6.0,15.0,3.0,60.0,2.0,40.0,44.0,2.0,11.0,3.0,62.0,1500000.0,Real Zaragoza
18,Carlos Soler,2223,ESP,Spain,MF,Central Midfield,25,3,3,242.0,2.7,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.9,0.2,0.2,0.4,3.0,13.0,7.0,0.37,0.00,0.37,0.00,0.00,0.35,0.08,0.43,0.06,0.14,4.0,1.0,25.0,1.49,0.37,0.0,0.0,30.0,3.0,0.04,0.1,-0.2,92.0,123.0,74.8,1388.0,378.0,52.0,59.0,88.1,23.0,28.0,82.1,9.0,24.0,37.5,0.2,-0.2,4.0,5.0,4.0,0.0,102.0,21.0,0.0,1.0,16.0,0.0,11.0,5.0,5.0,0.0,0.0,3.0,6.0,2.23,5.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,2.0,66.7,1.0,1.0,0.0,2.0,2.0,0.0,143.0,2.0,4.0,1.0,50.0,0.0,0.0,83.0,0.0,2.0,1.0,98.0,25000000.0,West Ham United
12,Cenk Özkacar,2223,TUR,Türkiye,DF,Centre-Back,21,17,14,1365.0,15.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.8,0.8,0.2,1.0,23.0,61.0,2.0,0.00,0.00,0.00,0.00,0.00,0.05,0.02,0.07,0.05,0.07,11.0,2.0,18.2,0.73,0.13,0.0,0.0,12.8,0.0,0.07,-0.8,-0.8,646.0,769.0,84.0,12014.0,4500.0,274.0,315.0,87.0,308.0,341.0,90.3,60.0,91.0,65.9,0.3,-0.2,4.0,54.0,3.0,1.0,744.0,23.0,0.0,16.0,3.0,2.0,0.0,0.0,0.0,0.0,2.0,8.0,18.0,1.19,15.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,27.0,20.0,17.0,10.0,0.0,14.0,73.7,5.0,12.0,12.0,39.0,47.0,4.0,908.0,85.0,16.0,10.0,100.0,0.0,0.0,522.0,0.0,8.0,4.0,549.0,5000000.0,Real Valladolid CF
35,Cenk Özkacar,2324,TUR,Türkiye,DF,Centre-Back,22,23,18,1598.0,17.8,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.3,0.3,0.1,0.3,17.0,56.0,5.0,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.02,0.01,0.02,6.0,3.0,50.0,0.34,0.17,0.0,0.0,14.3,0.0,0.04,-0.3,-0.3,698.0,860.0,81.2,13108.0,4662.0,265.0,295.0,89.8,364.0,398.0,91.5,58.0,135.0,43.0,0.3,-0.1,1.0,48.0,3.0,0.0,810.0,48.0,1.0,9.0,6.0,21.0,0.0,0.0,0.0,0.0,2.0,15.0,13.0,0.73,12.0,1.0,0.0,0.0,0.0,2.0,0.11,2.0,0.0,0.0,0.0,0.0,0.0,26.0,15.0,22.0,4.0,0.0,15.0,65.2,8.0,11.0,18.0,44.0,84.0,1.0,1035.0,109.0,12.0,3.0,37.5,4.0,50.0,532.0,0.0,4.0,3.0,617.0,4000000.0,Real Valladolid CF
78,Charlie Perez,2223,ESP,Spain,GK,Goalkeeper,20,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [12]:
df_with_market_values['Nation'].value_counts().head(3)

Nation
ESP    47
FRA     4
BRA     4
Name: count, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_with_market_values['Nation'] = le.fit_transform(df_with_market_values['Nation'])
df_with_market_values['Position'] = le.fit_transform(df_with_market_values['Position'])

/var/folders/1h/14p077t95lbcy6x5tc6bqx3h0000gn/T/ipykernel_53065/1260227516.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Nation'] = le.fit_transform(df_with_market_values['Nation'])
/var/folders/1h/14p077t95lbcy6x5tc6bqx3h0000gn/T/ipykernel_53065/1260227516.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Position'] = le.fit_transform(df_with_market_values['Position'])


In [22]:
correlation_with_market_value = df_with_market_values.corrwith(df_with_market_values['MarketValueEuro'])

print(correlation_with_market_value.sort_values(ascending=False).head(11))

MarketValueEuro    1.000000
Starts             0.718316
90s                0.693411
Min                0.693269
Cmp.3              0.645374
PrgDist            0.641521
Att.3              0.638499
Def_Pen            0.621067
MP                 0.609923
TotDist            0.591553
Carries            0.568287
dtype: float64


### 🔟 Top 10 Features Most Correlated with Market Value

- **Starts (0.718)** — Indicates how often a player is trusted to start matches; more starts = more value.
- **90s (0.693)** — Reflects total playing time in 90-minute equivalents; regular play boosts visibility and importance.
- **Min (0.693)** — Total minutes played; more minutes typically means more experience and trust from coaches.
- **Cmp.3 (0.645)** — Completed passes in the attacking third; shows involvement in crucial offensive phases.
- **PrgDist (0.642)** — Total progressive carrying distance; highlights ability to move the ball forward effectively.
- **Att.3 (0.638)** — Passes attempted in the attacking third; shows a player’s role in build-up and attacking moves.
- **Def_Pen (0.621)** — Defensive actions inside the penalty area; key for evaluating defenders’ last-line contributions.
- **MP (0.610)** — Matches played; simple yet powerful indicator of consistency and fitness.
- **TotDist (0.592)** — Total distance covered while carrying the ball; proxies physical activity and involvement.
- **Carries (0.568)** — Number of ball carries; emphasizes players who actively retain and progress possession.


- Nation and Position don't have an impact on Market Value

### Interrogations:
- When scraping data from other teams to build a regression model to predict a player's next year's market value, I believe we should consider the ranking of the team. 
- A player for Valencia might not be value the same as a Real Madrid player even with the same performance.
- That's because Real Madrid plays bigger competitions and is more likely to win titles which boosts the player's exposure and therefore the market price.
- number of competitions played as well

## Other spanish teams to scrape for ML:

- https://fbref.com/en/squads/53a2f082/2024-2025/Real-Madrid-Stats
- https://fbref.com/en/squads/206d90db/Barcelona-Stats
- https://fbref.com/en/squads/ad2be733/Sevilla-Stats
- https://fbref.com/en/squads/db3b9613/Atletico-Madrid-Stats
- https://fbref.com/en/squads/2b390eca/Athletic-Club-Stats
- https://fbref.com/en/squads/2a8183b3/Villarreal-Stats
- https://fbref.com/en/squads/e31d1cd9/Real-Sociedad-Stats
- https://fbref.com/en/squads/fc536746/Real-Betis-Stats

---

# Merge all player data

In [3]:
import sys
import os
from pathlib import Path
sys.path.append("..")
from src.preprocessing.merge_player_data import run_merge_pipeline

teams_list: list[str] = [
    "Real Madrid CF",
    "FC Barcelona", 
    "Atlético Madrid",
    "Sevilla FC", 
    "Athletic Club",
    "Villarreal CF",
    "Real Sociedad",
    "Real Betis",
    "Valencia CF",
]

seasons_list: list[str] = ["2223", "2324", "2425"]

for team_name in teams_list:
    print(f"Processing {team_name}...")
    
    # Create directory structure
    team_directory: Path = Path(f"../data/interim/{team_name}/merged")
    team_directory.mkdir(parents=True, exist_ok=True)
    
    # Run merge pipeline for current team
    merge_result: tuple = run_merge_pipeline(team_name, seasons_list)
    df_all_players = merge_result[0] if isinstance(merge_result, tuple) else merge_result
    
    print(f"Merge completed for {team_name}!")
    print(f"Total merged rows: {df_all_players.shape[0]}")
    print(f"Total columns: {df_all_players.shape[1]}")
    
    # Remove system/total rows
    player_column_filled = df_all_players['Player'].fillna('Unknown')
    df_clean = df_all_players[~player_column_filled.str.contains('Total|Squad|Opponent', case=False)]

    # Only players with market values
    df_with_market_values = df_all_players[df_all_players['MarketValueEuro'].notna()]
    
    # Clean nation column
    df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]
    
    # Save processed data
    clean_filename: str = f"{team_name.replace(' ', '_').lower()}_clean.csv"
    market_values_filename: str = f"{team_name.replace(' ', '_').lower()}_market_values.csv"
    
    df_clean.to_csv(team_directory / clean_filename, index=False)
    df_with_market_values.to_csv(team_directory / market_values_filename, index=False)
    
    print(f"Saved {clean_filename} and {market_values_filename} to {team_directory}")
    print("-" * 50)

print("All teams processed successfully!")


Processing Real Madrid CF...
Current working directory: /Users/markuskuehnle/Documents/projects/talent-value-forecast/notebooks
Found project root: /Users/markuskuehnle/Documents/projects/talent-value-forecast
Trying possible paths: ['/Users/markuskuehnle/Documents/projects/talent-value-forecast/data', '../data', '../../data', '/Users/markuskuehnle/Documents/projects/talent-value-forecast/data', '/Users/markuskuehnle/Documents/projects/data', 'data']
Checking path: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data - exists: True
Found data directory: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data
Looking for FBref directory: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data/interim/Real Madrid CF/fbref
Looking for Transfermarkt file: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data/interim/Real Madrid CF/transfermarkt/real_madrid_cf_2020_2024.csv
Attempting fuzzy matching for 41 unmatched players...
Fuzzy match: 'v

/var/folders/0z/26cwjfqx4fj73hndqg4mkgl80000gn/T/ipykernel_8628/1532477885.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]
/var/folders/0z/26cwjfqx4fj73hndqg4mkgl80000gn/T/ipykernel_8628/1532477885.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]
/var/folders/0z/26cwjfqx4fj73hndqg4mkgl80000gn/T/ipykernel_8628/1532477885.py:44: SettingWithCopyWarning: 

Attempting fuzzy matching for 59 unmatched players...
Fuzzy match: 'ivan rakitić' -> 'ivan rakitic' (score: 92)
Fuzzy match: 'loïc bade' -> 'loïc badé' (score: 93)
Fuzzy match: 'marko dmitrović' -> 'marko dmitrovic' (score: 93)
Fuzzy match: 'loïc bade' -> 'loïc badé' (score: 93)
Fuzzy match: 'ivan rakitić' -> 'ivan rakitic' (score: 92)
Fuzzy match: 'marko dmitrović' -> 'marko dmitrovic' (score: 93)
Fuzzy match: 'dodi lukebakio' -> 'dodi lukébakio' (score: 96)
Fuzzy match: 'lucien agoume' -> 'lucien agoumé' (score: 96)
Fuzzy match: 'alejo véliz' -> 'alejo veliz' (score: 95)
Fuzzy match: 'dodi lukebakio' -> 'dodi lukébakio' (score: 96)
Fuzzy match: 'loïc bade' -> 'loïc badé' (score: 93)
Fuzzy match: 'lucien agoume' -> 'lucien agoumé' (score: 96)
Fuzzy match: 'ruben vargas' -> 'rubén vargas' (score: 96)
Dropped Nation_* columns: ['Nation_player_shooting', 'Nation_player_passing', 'Nation_player_passing_types', 'Nation_player_gca', 'Nation_player_defense', 'Nation_player_possession']
Dropp

/var/folders/0z/26cwjfqx4fj73hndqg4mkgl80000gn/T/ipykernel_8628/1532477885.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]
/var/folders/0z/26cwjfqx4fj73hndqg4mkgl80000gn/T/ipykernel_8628/1532477885.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]
/var/folders/0z/26cwjfqx4fj73hndqg4mkgl80000gn/T/ipykernel_8628/1532477885.py:44: SettingWithCopyWarning: 

Attempting fuzzy matching for 89 unmatched players...
Fuzzy match: 'mohamed ali cho' -> 'mohamed-ali cho' (score: 100)
Fuzzy match: 'alex sola' -> 'álex sola' (score: 94)
Fuzzy match: 'roberto navarro' -> 'robert navarro' (score: 97)
Fuzzy match: 'ander martin' -> 'ander martín' (score: 96)
Fuzzy match: 'jonathan gomez' -> 'jonathan gómez' (score: 96)
Dropped Nation_* columns: ['Nation_player_shooting', 'Nation_player_passing', 'Nation_player_passing_types', 'Nation_player_gca', 'Nation_player_defense', 'Nation_player_possession']
Dropped Age columns: ['Age_player_shooting', 'Age_player_passing', 'Age_player_passing_types', 'Age_player_gca', 'Age_player_defense', 'Age_player_possession', 'Age_y', 'Age']
Renamed 'Age_x' to 'Age'
Converted 'Age' to integer (NaN values set to -1)
Removed 43 duplicate columns with _player_* pattern
Removed columns: ['Player_player_shooting', 'Pos_player_shooting', '90s_player_shooting', 'Gls_player_shooting', 'PK_player_shooting', 'PKatt_player_shooting', 

/var/folders/0z/26cwjfqx4fj73hndqg4mkgl80000gn/T/ipykernel_8628/1532477885.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]
